# Regression Trees

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import re
import numpy as np

In [ ]:
df = pd.read_csv('bases/auto-mpg.csv')

In [ ]:
df.head()

In [ ]:
df['horsepower'].unique()

In [ ]:
# Eliminando valores '?'
# re.sub('?', np.nan, df['horsepower'])
# df['horsepower'].fillna(df['horsepower'].median(), inplace=True)
df = df[df['horsepower']!='?']

In [ ]:
features = ['horsepower', 'weight']
# Garantindo que estão no formato correto
for ft in features:
    df[ft] = pd.to_numeric(df[ft])
# Construindo matriz de features e vetor de true labels
X = df[features]
y = df['mpg'].values.tolist()

In [ ]:
# Instanciando
reg = DecisionTreeRegressor(max_depth=2, min_samples_split=3)

In [ ]:
# Fit
reg.fit(X,y)

In [ ]:
# Visualizando
print(tree.export_text(reg))

In [ ]:
#predict
reg.predict([['300','4000']])

# Prunning

In [ ]:
# vamos usar um dataset diferente para entender o processo de prunning
from sklearn.datasets import load_breast_cancer
X, y = load_breast_cancer(return_X_y=True)

In [ ]:
#separando o arquivo em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
#Calculando Cost Complecity Pruning
# Aqui, vou usar Decision Tree Classifier, voltado para classificação, apenas para fins didáticos. O processo é o mesmo para regressão
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
path = clf.cost_complexity_pruning_path(X_train,y_train)

In [ ]:
path

In [ ]:
# obtendo valores de alpha e impureza
ccp_alphas, impurities = path.ccp_alphas, path.impurities

In [ ]:
# plot
fig, ax = plt.subplots()
ax.plot(ccp_alphas[:-1], impurities[:-1], marker="o", drawstyle="steps-post")
ax.set_xlabel("effective alpha")
ax.set_ylabel("total impurity of leaves")
ax.set_title("Total Impurity vs effective alpha for training set")

In [ ]:
# Treinando a árvore com diferentes valores de alpha
clfs = []
for ccp_alpha in ccp_alphas:
    clf = DecisionTreeClassifier(random_state=0, ccp_alpha=ccp_alpha)
    clf.fit(X_train, y_train)
    clfs.append(clf)
print(
    "Number of nodes in the last tree is: {} with ccp_alpha: {}".format(
        clfs[-1].tree_.node_count, ccp_alphas[-1]
    )
)

In [ ]:
# eliminando o último elemento de clfs e ccp_alphas por que representa a árvore com apenas uma folha
clfs = clfs[:-1]
ccp_alphas = ccp_alphas[:-1]

In [ ]:
clfs

In [ ]:
#Obtendo os scores para cada valor de aplha
train_scores = [clf.score(X_train, y_train) for clf in clfs]
test_scores = [clf.score(X_test, y_test) for clf in clfs]

In [ ]:
#plotando
fig, ax = plt.subplots()
ax.set_xlabel("alpha")
ax.set_ylabel("accuracy")
ax.set_title("Accuracy vs alpha for training and testing sets")
ax.plot(ccp_alphas, train_scores, marker="o", label="train", drawstyle="steps-post")
ax.plot(ccp_alphas, test_scores, marker="o", label="test", drawstyle="steps-post")
ax.legend()
plt.show()

# Decision Trees

In [ ]:
import os
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "decision_trees"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)


In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
 
iris = load_iris()
X = iris.data[:, 2:] # petal length and width
y = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X, y)

In [ ]:
iris

É possível visualizar a árvore de decisão treinada usando o método export_graphviz(). 
OBS: Verifique se o pacote está instalado

In [ ]:
from sklearn.tree import export_graphviz

def image_path(fig_id):
    return os.path.join(IMAGES_PATH, fig_id)

export_graphviz(
        tree_clf,
        out_file=image_path("iris_tree.dot"),
        feature_names=iris.feature_names[2:],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )

Agora, vá em File -> New -> Terminal:

1. Navegue até a pasta onde o arquivo ires_tree.dot foi gerado 
2. Execute o seguinte código: dot -Tpng .\iris_tree.dot -o iris_tree.png

Isso vai gerar um arquivo png que mostrará como ficou nossa árvore. Vou adicioná-la aqui para facilitar nosso entendimento

<img src="images/decision_trees/iris_tree.png" />

### Como o classificador realiza novas predições?

- Começando pela raiz, verifique se o comprimento da pétala é menor que 2.45cm.

    - Se for, vá para o nó da esquerda. Como é um nó folha, a árvore prediz que a classe é setosa
    
    - Se não, vá para o nó da direita. Verifique se a largura da folha é menor que 1.75cm.
    
        - Se for, vá para o nó folha da esquerda. A flor será classificada como versicolor
        
        - Se não, vá para o nó folha da direita. A flor será classificada como virginica

### Desicion Boundaries
É possível verificar as fronteiras de decisão que o arquivo encontrou na construção da árvore. O código abaixo nos fornece uma visualização. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

def plot_decision_boundary(clf, X, y, axes=[0, 7.5, 0, 3], iris=True, legend=False, plot_training=True):
    x1s = np.linspace(axes[0], axes[1], 100)
    x2s = np.linspace(axes[2], axes[3], 100)
    x1, x2 = np.meshgrid(x1s, x2s)
    X_new = np.c_[x1.ravel(), x2.ravel()]
    y_pred = clf.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    if not iris:
        custom_cmap2 = ListedColormap(['#7d7d58','#4c4c7f','#507d50'])
        plt.contour(x1, x2, y_pred, cmap=custom_cmap2, alpha=0.8)
    if plot_training:
        plt.plot(X[:, 0][y==0], X[:, 1][y==0], "yo", label="Iris setosa")
        plt.plot(X[:, 0][y==1], X[:, 1][y==1], "bs", label="Iris versicolor")
        plt.plot(X[:, 0][y==2], X[:, 1][y==2], "g^", label="Iris virginica")
        plt.axis(axes)
    if iris:
        plt.xlabel("Petal length", fontsize=14)
        plt.ylabel("Petal width", fontsize=14)
    else:
        plt.xlabel(r"$x_1$", fontsize=18)
        plt.ylabel(r"$x_2$", fontsize=18, rotation=0)
    if legend:
        plt.legend(loc="lower right", fontsize=14)

plt.figure(figsize=(8, 4))
plot_decision_boundary(tree_clf, X, y)
plt.plot([2.45, 2.45], [0, 3], "k-", linewidth=2)
plt.plot([2.45, 7.5], [1.75, 1.75], "k--", linewidth=2)
plt.plot([4.95, 4.95], [0, 1.75], "k:", linewidth=2)
plt.plot([4.85, 4.85], [1.75, 3], "k:", linewidth=2)
plt.text(1.40, 1.0, "Depth=0", fontsize=15)
plt.text(3.2, 1.80, "Depth=1", fontsize=13)
plt.text(4.05, 0.5, "(Depth=2)", fontsize=11)

plt.show()

A linha mais grossa representa a fronteira de decisão do nó raiz: petal length = 2.45cm. Como a área à esquerda é pura, não há mais splits. Entretanto, a área à direita é impura, então o nó é dividido quando petal eidth = 1.75cm. Visto que max_depth=2, Decision Tree para aqui. 

### Estimando as probabilidades das classes

Uma Árvore de Decisão pode estimar a probabilidade de uma instância pertencer a uma classe particular. Suponha que uma flor tenha 5cm de comprimento e 1.5cm de largura. O nó correspondente seria o Depth-2 a esquerda. Nesse caso, a Árvore retornaria as seguintes probabilidades:

- 0% parra *Iris Setosa* (0/54)
- 90.7% para *Iris Versicolor* (49/54)
- 9.3% para Iris Virginica* (5/54)

In [ ]:
tree_clf.predict_proba([[5, 1.5]])

E se você solicitar a predição da classe, o algoritmo retornará *Iris versicolor* (classe 1), visto que ela possui a maior probabilidade:

In [ ]:
tree_clf.predict([[5, 1.5]])

### Regularização dos Hiperparâmetros

Árvores de decisão assumem poucas suposições acerca dos dados de treinamento (ao contrário dos modelos lineares, que assumem que os dados são lineares). Se deixada sem restrição, a árvore irá se adaptar aos dados de treino, dando um fit muito próximo (na verdade, overfit). 

Tais modelos são chamados não-paramétricos, mas não porque não possuem parâmetros (ou hiperparâmetros), mas porque o número de parâmetros não é determinado antes do treino.

PAra evitar overfitting, é preciso restringir a liberdade da Árvore de Decisão durante o treinamento. Isto é chamado de regularização. Alguns hiperparâmetros que podem ser ajustados para evitar overfitting numa Árvore de decisão:

- max_depth (profundidade máxima da árvore - delimita o número de splits)
- min_samples_split (número mínimo de amostras que um nó deve ter antes do split)
- min_sampes_leaf (número mínimo de amostras que um nó folha deve ter)

O código abaixo apresenta duas Árvores treinadas no dataset moons. A da esquerda foi treinada com hiperparâmetros padrões (sem restrições), enquanto a da direita foi regularizada (min_samples_leaf=4). 

Podemos observar que o modelo da esquerda está dando overfiting e que o modelo da direita irá generalizar melhor. 

In [ ]:
from sklearn.datasets import make_moons
Xm, ym = make_moons(n_samples=100, noise=0.25, random_state=53)

deep_tree_clf1 = DecisionTreeClassifier(random_state=42)
deep_tree_clf2 = DecisionTreeClassifier(min_samples_leaf=4,max_depth=3 random_state=42)
deep_tree_clf1.fit(Xm, ym)
deep_tree_clf2.fit(Xm, ym)

fig, axes = plt.subplots(ncols=2, figsize=(10, 4), sharey=True)
plt.sca(axes[0])
plot_decision_boundary(deep_tree_clf1, Xm, ym, axes=[-1.5, 2.4, -1, 1.5], iris=False)
plt.title("No restrictions", fontsize=16)
plt.sca(axes[1])
plot_decision_boundary(deep_tree_clf2, Xm, ym, axes=[-1.5, 2.4, -1, 1.5], iris=False)
plt.title("min_samples_leaf = {}".format(deep_tree_clf2.min_samples_leaf), fontsize=14)
plt.ylabel("")

plt.show()